In [ ]:
%env OPENAI_API_KEY=keyhere

In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

import asyncio
from kagent.tools.docs import search_documentation

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

cncf_docs_agent = AssistantAgent(
    "kubernetes_docs_agent",
    description="This agent allows you to get data from the CNCF docs databases.",
    tools=[search_documentation],
    model_client=model_client,
    system_message="""
    You are a support agent.

    You have access to several tools:
    - 'searchDocumentation' to search in the documentation.

    Execute all the following steps:
    1. Product identification
    - Check if you know what product the question is about (Only supported products are: 'kubernetes', 'istio', 'argo', 'helm', 'prometheus')
    - If it's not the case, ask what the product is

    2. Information Gathering
    - Search in the documentation for information related to the question the user has submitted

    3. Answer the question
    - Use all the information you gathered to provide a valuable answer to the user
    - Provide links to the documentation whenever possible`,
    """,
)

user_proxy = UserProxyAgent("user_proxy", input_func=input)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


team = RoundRobinGroupChat(
    [
        cncf_docs_agent,
    ],
    termination_condition=termination,
)

# task = "How do I do a canary upgrade of a pod in kubernetes using argo rollouts?"
# task = "How to I install a helm chart in it doesn't exist or upgrade it if it does?"
task = "Write a Prometheus query to check if Istio requests are using mTLS?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
Write a Prometheus query to check if Istio requests are using mTLS?
---------- kubernetes_docs_agent ----------
[FunctionCall(id='call_w0brjmzNoEm9BZc7BP7N7Ewq', arguments='{"query":"check if Istio requests are using mTLS","product":"prometheus","version":"","limit":5}', name='search_documentation')]
---------- kubernetes_docs_agent ----------
[FunctionExecutionResult(content='[{"distance": 1.0775805711746216, "content": "\\nTLS and authentication are currently being rolled out to the Prometheus, Alertmanager, and the official exporters. Adding this support will make it easier for people to deploy Prometheus components securely without requiring a reverse proxy to add those features externally.\\n\\n"}, {"distance": 1.0935146808624268, "content": "probe_tls_version_info{version=\\"TLS 1.3\\"} 1\\n```\\n\\nYou can see that the probe was successful and get many useful metrics, like latency by phase, status code, ssl status or certificate expiry in [Unix time](h

In [ ]:
print(team.dump_component().model_dump_json(indent=2))